## Spell checking

This workbook holds some exploration of the spell checking code.

Our model for spell checking requires four main steps: 

1. Selecting which known word is the highest priority. 
1. Coming up with the candidates that _could_ be the correct spelling of an unknown word. 
1. The language model: the probability of a given candidate.
1. The error model: Given that $c$ is correct, how probable is $P(w|c)$? 

There are some implied other tasks we'll need to take care of:

* Discovering if a word is known.
* Estimating the frequency of a word in a corpus (this is the language model). 
* Finding potential "single edit" candidates. 
* Finding potential "double edit" candidates.


In [ ]:
import nltk
import re
from collections import Counter

---

### Finding known words

Let's start by exploring the notion of "known words". The file `wsj_with_errors.txt` has a bunch of text in it with errors introduced. 

1. Ingest this file one word at a time.
1. Remove punctuation and numbers.
1. Cast to lowercase.
1. Take every word that’s not in the NLTK word corpus and write it to a file. 


In [ ]:
nltk_words = {w.lower() for w in nltk.corpus.words.words()}
word_regex = re.compile(r'^[a-z]+$')

In [ ]:
# your code here. 
unknown = set()
with open("wsj_with_errors.txt",'r') as infile :
    for line in infile :
        line = [w.lower() for w in line.strip().split()]
        line = [w for w in line if word_regex.search(w)]

        for word in set(line) - nltk_words :
            unknown.add(word)

                
with open("in_wsj_not_nlkt.txt",'w') as outfile :
    for word in unknown :
        outfile.write(word + "\n")

print(len(unknown))

The file `big.txt` has 1.1M words in it. Repeat the above steps but use this as our set of known words. Compare the results. How much of a difference does the big corpus provide? 

In [ ]:
all_big_words = list() # useful below

with open("big.txt") as infile :
    for line in infile :
        line = [w.lower() for w in line.strip().split()]
        line = [w for w in line if word_regex.search(w)]
        
        for word in line :
            all_big_words.append(word)

            
big_words = set(all_big_words) 

In [ ]:
# more of your code here. 
unknown = set()
with open("wsj_with_errors.txt",'r') as infile :
    for line in infile :
        line = [w.lower() for w in line.strip().split()]
        line = [w for w in line if word_regex.search(w)]

        for word in set(line) - big_words :
            unknown.add(word)

with open("in_wsj_not_big.txt",'w') as outfile :
    for word in unknown :
        outfile.write(word + "\n")

print(len(unknown))

---

### Word Frequency

Take the `big.txt` file and create a variable called `WORDS` that is a frequency distribution of the words in `big.txt`. Again, remove punctuation and numbers and cast to lowercase. (`WORDS` should be a `Counter` object.) Answer some questions with it:

* How many times does the word "the" appear? 
* What fraction of the words are "the"?
* What is the 30th most common word?
* How many words appear a single time? 

In [ ]:
# your code here
WORDS = Counter(all_big_words)

print(WORDS['the']) # 78172
print(WORDS['the']/sum(WORDS.values())) # 0.0853
print(WORDS.most_common(30)) # they is #30
print(len([w for w, c in WORDS.items() if c == 1])) # 7994


In order to do this efficiently, we're going to need some functions. Write a function called `known` that takes a list of words and returns a _set_ of the words in that list which are known.

In [ ]:
def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)
    
    
    
assert(known(['hello','help','halp'])=={'hello','help'})
assert(known(['a','bunch','of','words'])=={'a','bunch','of','words'})
assert(known(['0ops','l33t','qwjibo'])==set())

Now write a function called `P` that takes a word and returns that word's fraction in the corpus. So `P("the")` should return `0.08533967969781989`. 

In [ ]:
def P(word) :
    N = sum(WORDS.values())
    return (WORDS[word]/N)


assert(P("the")==0.08533967969781989)
assert(P("happy")==0.00018012903789259941)
assert(P("fall")==0.00011571926070676085)

---

### Finding Single Edit Mistakes

Our error model requires us to find single edit mistakes in a word. As we covered in the lecture, there are four types of single-edit mistakes: deletions, transpositions, replacements, and insertions. 

To start, we follow a clever approach from Norvig. What does this next cell do?  

In [ ]:
word = "monkey"
splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
splits

Take this hint and try to make all *insertions* first. That's when we just include a random letter somewhere in the word. For the word `monkey`, how many insertions are there? 

In [ ]:
letters = 'abcdefghijklmnopqrstuvwxyz'

insertions = [L + c + R for L,R in splits for c in letters]
print(len(insertions))

Up next, let's try to make the replacements. Now we're replacing a letter with a random letter. 

In [ ]:
replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]

Now let's make deletions and transpositions. These are a bit trickier. 

In [ ]:
deletes    = [L + R[1:]               for L, R in splits if R]
transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]

Okay, now write a function that takes a word as input and returns a _set_ all of the words that are one edit away. Call this `edits1`.  

In [ ]:
def edits1(word) :
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    
    return set(deletes + transposes + replaces + inserts)


assert(len(edits1('a'))==78)
assert(len(edits1('apple'))==284)
assert(len(edits1('applegate'))==492)
assert(len(edits1('ambidexterous'))==702)

It's easy to get a lot of possible edits. How many of the above are known? Run them through your previous function. Also try out some other words to make sure your functions are working properly

In [ ]:
known(edits1("monkey"))

In [ ]:
known(edits1("apple"))

In [ ]:
known(edits1("text"))

---

### Spell Checking

We're close! The mechanics of our algorithm are as follows: 

1. Start with a word.
1. If it's known, return that word. 
1. If it's not known, calculate the known words one edit distance away. 
1. If there are known words at one edit distance, return the highest probability one.
1. Else, calculate the known words at two edit distances away. 
1. If there are any, return the one with the highest probability. 
1. Otherwise, return the word itself. 

In the cell below, create a function called `correction` that carries out these steps. It should take a word as the argument and will make use of your `WORDS` variable. 


In [ ]:
def correction(word) :

    if known([word]) :
        return(word)
    else :
        ed_1 = edits1(word)
        candidates = known(ed_1)
        if candidates :
            return(max(candidates, key=P))
        else : 
            ed_2 = (e2 for e1 in ed_1 for e2 in edits1(e1))
            candidates = known(ed_2)
            if candidates :
                return(max(candidates, key=P))
            else :
                return(word)
            

assert(correction("thew")=="the")
assert(correction("tst")=="test")
assert(correction("mano")=="man")
assert(correction("myxomatosis")=="myxomatous")